# 连接

In [1]:
import numpy as np
import pandas as pd

## 关系型连接

### 连接的基本概念

pandas提供了两种**关系型连接函数**，即`merge`和`join`。它们的参数`on`代表连接的主键，`how`代表连接的方式。

`how`参数有四种取值：`left`（左连接）、`right`（右连接）、`inner`（内连接）以及`outer`（外连接）。

### 值连接

基于一列或几列**取值**的连接可以通过`merge`函数实现。

In [2]:
# 生成示例
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],
                    'Age':[20,30]})
df2 = pd.DataFrame({'Name':['Si Li','Wu Wang'],
                    'Gender':['F','M']})

In [3]:
df1

,Name,Age
0,San Zhang,20
1,Si Li,30


In [4]:
df2

,Name,Gender
0,Si Li,F
1,Wu Wang,M


In [5]:
# 以Name为键，左连接
df1.merge(df2, on = 'Name', how = 'left')

,Name,Age,Gender
0,San Zhang,20,NaN
1,Si Li,30,F


若待连接的两个表的键名称不同，可以通过`left_on`和`right_on`参数来指定。

In [6]:
df1 = pd.DataFrame({'df1_name':['San Zhang','Si Li'],
                    'Age':[20,30]})
df2 = pd.DataFrame({'df2_name':['Si Li','Wu Wang'],
                     'Gender':['F','M']})

In [7]:
df1

,df1_name,Age
0,San Zhang,20
1,Si Li,30


In [8]:
df2

,df2_name,Gender
0,Si Li,F
1,Wu Wang,M


In [9]:
# 以第一列为主键，左连接
df1.merge(df2, left_on = 'df1_name', right_on = 'df2_name', how = 'left')

,df1_name,Age,df2_name,Gender
0,San Zhang,20,NaN,NaN
1,Si Li,30,Si Li,F


若待连接的量表存在名称相同的变量，可以通过指定`suffixes`参数来进行区分。

In [10]:
# 语文成绩
df1 = pd.DataFrame({'Name':['San Zhang'],'Grade':[70]})
# 数学成绩
df2 = pd.DataFrame({'Name':['San Zhang'],'Grade':[80]})

In [11]:
# 指定下标
df1.merge(df2, on = 'Name', how = 'left', suffixes = ['_Chinese','_Math'])

,Name,Grade_Chinese,Grade_Math
0,San Zhang,70,80


可向`on`参数多个变量名组成的列表，从而定义复合主键。

In [12]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['two', 'one']})

In [13]:
# 以姓名和班级定义复合主键，左连接
df1.merge(df2, on = ['Name','Class'], how = 'left')

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


`merge`函数提供了`validate`参数来检查连接的**唯一性模式**，其取值包括：`1:1`（一对一连接）、`1:m`（一对多连接）和`m:1`（多对一连接）。

#### 练一练1

In [14]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Age':[20, 21],
                    'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                    'Gender':['F', 'M'],
                    'Class':['one', 'one']})

In [15]:
df1.merge(df2, on = ['Name','Class'], how = 'left', validate = '1:m')

,Name,Age,Class,Gender
0,San Zhang,20,one,F
1,San Zhang,20,one,M
2,San Zhang,21,two,NaN


In [16]:
try:
    df1.merge(df2, on = ['Name','Class'], how = 'left', validate = 'm:1')
except Exception as e:
    print(e)

Merge keys are not unique in right dataset; not a many-to-one merge


### 索引连接

pandas利用`join`函数来实现**索引**连接，该连接方式将索引当作主键。其中，`on`参数为索引名。

In [17]:
df1 = pd.DataFrame({'Age':[20,30]},
                    index=pd.Series(['San Zhang','Si Li'],name='Name'))
df2 = pd.DataFrame({'Gender':['F','M']},
                   index=pd.Series(['Si Li','Wu Wang'],name='Name'))

In [18]:
df1.join(df2, how = 'left')

,Age,Gender
Name,,
San Zhang,20,NaN
Si Li,30,F


利用`join`函数进行连接时，若出现名称相同的列名，则应通过参数`lsuffix`和`rsuffix`来分别指定左右下标。

当待连接的表存在多级索引时，相当于默认为`join`指定了复合主键。

## 方向连接

### concat

如果只是想当纯地进行两个表之间的横向或纵向拼接，可以考虑使用`concat`函数。关键参数`axis`代表了拼接方向，取值0（默认）代表纵向拼接，1代表横向拼接。

利用`concat`进行纵（横）向拼接时，表格会根据列（行）对其。相关的参数为`join`：若`join=outer`（默认），拼接时会保留所有的列（行），并将不存在的值记为缺失；若`join=inner`，拼接时只保留两表共有的列（行）。根据此规则，在进行横向合并时，最好先通过`reset_index`来**恢复**默认整数索引，避免发生错误的对齐。

此外`concat`函数还可指定参数`keys`，它可以通过生成索引来区分合并表中数据的来源。

In [19]:
# 示例
df1 = pd.DataFrame({'Name':['San Zhang','Si Li'],'Age':[20,21]})
df2 = pd.DataFrame({'Name':['Wu Wang'],'Age':[21]})
# 指定keys，来区分来源
pd.concat([df1, df2], keys=['one', 'two'])

Name  Age
one 0  San Zhang   20
    1      Si Li   21
two 0    Wu Wang   21

### 序列与表的合并

若想在原表中**新增一行**，则可以使用`append`函数。需要注意的是，通常会指定参数`ignore_index = True`来实现对新增行索引的**自动标号**，否则必须为Series定义`name`属性。

In [20]:
# 示例
# 定义Series作为新增的行
s = pd.Series(['Wu Wang', 21], index = df1.columns)
# 添加
df1.append(s, ignore_index = True)

,Name,Age
0,San Zhang,20
1,Si Li,21
2,Wu Wang,21


若想在原表中**新增一列**，则可以使用`assign`函数。另外一种方法是直接定义`df['new_col']=...`。区别在于，后者直接在原表上改动，而前者则会返回一个临时的副本。

In [21]:
# 示例
s = pd.Series([80, 90])
# 新增一列，命名为Grade
df1.assign(Grade=s)

,Name,Age,Grade
0,San Zhang,20,80
1,Si Li,21,90


## 类连接操作

### 比较

可以通过`compare`函数来比较两张表的内容，如以下案例所示：

In [22]:
# 定义示例
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'Name':['San Zhang', 'Li Si', 'Wu Wang'],
                    'Age':[20, 21 ,21],
                    'Class':['one', 'two', 'Three']})
# 进行比较
df1.compare(df2)

Name         Class       
    self  other   self  other
1  Si Li  Li Si    NaN    NaN
2    NaN    NaN  three  Three

可以看到，`compare`函数返回的是`df1`（`self`）和`df2`（`other`）两张表取值不同的元素及其所在的行列，取值相同处记为`NaN`。若想返回完整表格的对比情况，可设置参数`keep_shape=True`：

In [23]:
# 返回对比结果，保持原有形状
df1.compare(df2, keep_shape=True)

Name         Age        Class       
    self  other self other   self  other
0    NaN    NaN  NaN   NaN    NaN    NaN
1  Si Li  Li Si  NaN   NaN    NaN    NaN
2    NaN    NaN  NaN   NaN  three  Three

### 组合

`combine`函数能够实现两张表按照一定的规则进行组合。

In [24]:
# 定义函数，实现选出两表相同位置较小的元素，如果其中有缺失则为缺失
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1) # 参照s1，对s2的索引变形
    res = s1.where(s1<s2, s2) # 如果s1对应位置的值大于等于s2，就将该值替换为s2所在位置的值
    res = res.mask(s1.isna()) #如果res和s1对应位置上，后者为缺失值，则res该位置的值也缺失，这实际上是在对上一个步骤进行调整，因为NaN要比任何实际数字都大，但理论上来说我们无法与缺失值比较
    return res

In [25]:
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])

In [26]:
df1

,A,B,C
0,1,3,5
1,2,4,6


In [27]:
df2

,B,C,D
1,5,7,9
2,6,8,10


In [28]:
df1.combine(df2, choose_min)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


如果设置参数`overwrite = False`，则可以保留`df1`中未出现在`df2`的列（即`A`列）。

In [29]:
df1.combine(df2,choose_min,overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


#### 练一练2

In [30]:
# 只用把原函数体的第三行删去即可，也就是说如果相同位置，s1缺失且s2存在，则返回结果对应位置填充为s2值
def choose_min_adj(s1, s2):
    s2 = s2.reindex_like(s1) 
    res = s1.where(s1<s2, s2) 
    return res

In [31]:
df1.combine(df2, choose_min_adj)

,A,B,C,D
0,NaN,NaN,NaN,NaN
1,NaN,4.0,6.0,9.0
2,NaN,6.0,8.0,10.0


#### 练一练3

In [32]:
df1 = pd.DataFrame({'A':[1,2], 'B':[np.nan,np.nan]})
df2 = pd.DataFrame({'A':[5,6], 'B':[7,8]}, index=[1,2])

In [33]:
df1

,A,B
0,1,NaN
1,2,NaN


In [34]:
df2

,A,B
1,5,7
2,6,8


In [35]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,7.0
2,6.0,8.0


In [36]:
def cbf(s1, s2):
    s2 = s2.reindex_like(s1) 
    res = s1.mask(s1.isna(), s2) 
    return res

In [37]:
# 利用combine实现
df1.combine(df2,cbf)

,A,B
0,1.0,NaN
1,2.0,7.0
2,6.0,8.0


## 练习

### Ex1: 美国疫情数据集

由于涉及大量独立表格的合并，所以肯定得使用**循环**，为此需生成由文件名构成的列表，方便循环时读取。

In [38]:
import os

In [39]:
# 生成文件名列表
datalist = os.listdir('./data/us_report')
datalist = ['./data/us_report/' + x for x in datalist]

In [40]:
# 生成备用索引
date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2)+'-'+date.dt.day.astype('string').str.zfill(2)+'-'+'2020'
date = date.tolist()

先进行初步尝试。

In [41]:
df1 = pd.read_csv('./data/us_report/04-12-2020.csv')

In [42]:
df1 = df1.set_index('Province_State').loc['New York'][['Confirmed','Deaths','Recovered','Active']]

In [43]:
#pd.concat([df1, df2],1)
df1 = pd.DataFrame(df1).T.reset_index(drop = True)

In [44]:
df1

,Confirmed,Deaths,Recovered,Active
0,189033,9385,23887,179648


In [45]:
df2 = pd.read_csv('./data/us_report/04-13-2020.csv')

In [46]:
df2 = df2.set_index('Province_State').loc['New York'][['Confirmed','Deaths','Recovered','Active']]

In [47]:
df2 = pd.DataFrame(df2).T.reset_index(drop=True)

In [48]:
pd.concat([df1,df2],keys=date[:2]).reset_index(1,drop=True)

,Confirmed,Deaths,Recovered,Active
04-12-2020,189033,9385,23887,179648
04-13-2020,195749,10058,23887,185691


实现：

In [49]:
# 写循环
mylist = []

for df in datalist:
    z = pd.read_csv(df)
    z = z.set_index('Province_State').loc['New York'][['Confirmed','Deaths','Recovered','Active']]
    z = pd.DataFrame(z).T.reset_index(drop=True)
    mylist.append(z)

report = pd.concat(mylist,keys=date).reset_index(1,drop=True)

In [50]:
report.head()

,Confirmed,Deaths,Recovered,Active
04-12-2020,189033,9385,23887,179648
04-13-2020,195749,10058,23887,185691
04-14-2020,203020,10842,23887,192178
04-15-2020,214454,11617,23887,202837
04-16-2020,223691,14832,23887,208859


与我自己的思路相比，答案的实现方式更加紧凑、简洁，非常值得学习：

In [51]:
L = []
# 写循环
for d in date:
    # 因为报表的命名格式是固定的，所以不需要用os来读取；而且参考答案的方式不会出现错误
    df = pd.read_csv('./data/us_report/' + d + '.csv', index_col = 'Province_State')
    data = df.loc['New York',['Confirmed','Deaths','Recovered','Active']]
    # 可以直接由to_frame方法实现转换
    L.append(data.to_frame().T)

res = pd.concat(L)
res.index = date
res.head()

,Confirmed,Deaths,Recovered,Active
04-12-2020,189033,9385,23887,179648
04-13-2020,195749,10058,23887,185691
04-14-2020,203020,10842,23887,192178
04-15-2020,214454,11617,23887,202837
04-16-2020,223691,14832,23887,208859


### Ex2: 实现join函数

这一题太复杂了，正在尝试理解答案的思路。